In [1]:
#importing libraries
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn
from collections import OrderedDict
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
#importing dataset
housing = pd.DataFrame(pd.read_csv("data/Housing.csv")) 
housing.head() 

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
#slicing out desired features from dataset
parameters = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking'] 
data_params = housing[parameters] 
t_u = housing[parameters] 
data_params.head() 
data_params.shape
housing_price=housing['price']
t_c=housing['price']
data_price=housing_price

In [4]:
t_u = torch.tensor(t_u.values,dtype=torch.float)
t_c = torch.tensor(t_c.values,dtype=torch.float)

In [5]:
#turning paramaters into a tensor
data = torch.tensor(data_params.values,dtype=torch.float)
data.shape

torch.Size([545, 5])

In [6]:
#converting cost to tensor
cost = torch.tensor(data_price.values,dtype=torch.float)
torch.unsqueeze(cost,0)
cost.shape

torch.Size([545])

In [7]:
#seperating data into 80/20 train/val split
n_samples = data.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train= data[train_indices]
t_c_train= cost[train_indices]
t_u_val = data[val_indices]
t_c_val = cost[val_indices]



t_cn_val = F.normalize(t_c_val,dim=0)
t_cn_train= F.normalize(t_c_train,dim=0)

t_un_val = F.normalize(t_u_val,dim=0)
t_un_train= F.normalize(t_u_train,dim=0)


In [8]:
import datetime

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()



def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) 
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) 
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() 
        optimizer.step()
            
        if epoch == 1  or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}, Validation loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train.item(), loss_val.item()))

In [9]:
#First model with following hidden layer size..
#    1st hidden layer = 8

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(5, 8)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(8, 1))
]))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)



training_loop(
    n_epochs = 300, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_cn_train,
    t_c_val = t_cn_val)

C:\Users\Jeromey\anaconda3\envs\RealTime\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Jeromey\anaconda3\envs\RealTime\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


2022-03-04 22:28:09.918132 Epoch 1, Training loss 0.028651388362050056, Validation loss 0.014564200304448605
2022-03-04 22:28:09.924138 Epoch 10, Training loss 0.01570279523730278, Validation loss 0.0063016293570399284
2022-03-04 22:28:09.931144 Epoch 20, Training loss 0.008127103559672832, Validation loss 0.002655675867572427
2022-03-04 22:28:09.937149 Epoch 30, Training loss 0.004288597963750362, Validation loss 0.0016993959434330463
2022-03-04 22:28:09.943154 Epoch 40, Training loss 0.0023459051735699177, Validation loss 0.0018503759056329727
2022-03-04 22:28:09.949160 Epoch 50, Training loss 0.001363735762424767, Validation loss 0.002378766192123294
2022-03-04 22:28:09.956166 Epoch 60, Training loss 0.0008676169672980905, Validation loss 0.002967244014143944
2022-03-04 22:28:09.962172 Epoch 70, Training loss 0.0006171820568852127, Validation loss 0.0034928845707327127
2022-03-04 22:28:09.968177 Epoch 80, Training loss 0.0004908245173282921, Validation loss 0.003920475486665964
2022

In [10]:
seq_model

for price, param in seq_model.named_parameters():
    print(price, param.shape)

hidden_linear.weight torch.Size([8, 5])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [11]:
list(seq_model.parameters())

[Parameter containing:
 tensor([[-0.3977, -0.2150,  0.2646,  0.1530,  0.1002],
         [ 0.2447,  0.2429, -0.1354,  0.3185, -0.1918],
         [ 0.2127, -0.0214, -0.1124,  0.4138,  0.1655],
         [ 0.2770, -0.0083, -0.0936, -0.3290, -0.4305],
         [-0.3736, -0.3984, -0.2914, -0.4154, -0.4441],
         [ 0.0920, -0.2390, -0.0269,  0.0157, -0.2983],
         [ 0.0879, -0.1470, -0.2385, -0.4022,  0.3040],
         [-0.4166,  0.1021,  0.4130,  0.2814,  0.0130]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0922,  0.2776, -0.1572, -0.3498, -0.3853, -0.0957, -0.2216,
          0.1180], requires_grad=True),
 Parameter containing:
 tensor([[ 0.1311,  0.1615, -0.0231,  0.0229,  0.1894,  0.2741, -0.0847,
          -0.1692]], requires_grad=True),
 Parameter containing:
 tensor([0.1307], requires_grad=True)]

In [12]:
seq_model.output_linear.bias

Parameter containing:
tensor([0.1307], requires_grad=True)

In [13]:
#Second model with following hidden layer size..
#    1st hidden layer =8
#    2nd hidden layer = 80
#    3rd hidden layer = 20
seq_model = nn.Sequential(
     nn.Linear(5, 8),
     nn.Tanh(),
     nn.Linear(8, 80),
     nn.Tanh(),
     nn.Linear(80, 20),
     nn.Tanh(),
     nn.Linear(20, 1))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)



training_loop(
    n_epochs = 300, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_cn_train,
    t_c_val = t_cn_val)

2022-03-04 22:28:10.184374 Epoch 1, Training loss 0.0035714362747967243, Validation loss 0.0016199146630242467
2022-03-04 22:28:10.195384 Epoch 10, Training loss 0.0014971073251217604, Validation loss 0.0015192347345873713
2022-03-04 22:28:10.207394 Epoch 20, Training loss 0.0006887711933813989, Validation loss 0.0020097861997783184
2022-03-04 22:28:10.221407 Epoch 30, Training loss 0.0004246427852194756, Validation loss 0.002489412436261773
2022-03-04 22:28:10.234419 Epoch 40, Training loss 0.0003386647440493107, Validation loss 0.0028282084967941046
2022-03-04 22:28:10.246430 Epoch 50, Training loss 0.0003107429656665772, Validation loss 0.003042487194761634
2022-03-04 22:28:10.259442 Epoch 60, Training loss 0.00030168716330081224, Validation loss 0.0031713994685560465
2022-03-04 22:28:10.271453 Epoch 70, Training loss 0.0002987523621413857, Validation loss 0.0032470047008246183
2022-03-04 22:28:10.284465 Epoch 80, Training loss 0.00029780136537738144, Validation loss 0.0032907444983

In [14]:
list(seq_model.parameters())

[Parameter containing:
 tensor([[ 0.1373, -0.0156,  0.0818,  0.0031, -0.3313],
         [-0.0875,  0.2662, -0.4049,  0.3106,  0.2989],
         [-0.3503, -0.4207,  0.4164,  0.1899,  0.3309],
         [ 0.1285, -0.2973, -0.3550,  0.1005, -0.0111],
         [-0.2541,  0.3821,  0.3074,  0.0941,  0.1438],
         [ 0.3906,  0.4204, -0.1065, -0.1923, -0.0641],
         [-0.3960, -0.1637,  0.2401, -0.3826, -0.3300],
         [ 0.4185, -0.1176,  0.0901, -0.2188, -0.2253]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.1394, -0.4427, -0.3844, -0.3718, -0.3950,  0.0229, -0.2412,
         -0.4430], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2562, -0.0044,  0.2494,  0.1027,  0.1480, -0.0035, -0.0149,
          -0.0917],
         [-0.2096,  0.2282, -0.1149,  0.0604, -0.2841, -0.0975, -0.2280,
           0.3529],
         [-0.2309, -0.3491, -0.0332,  0.0562,  0.0495, -0.0805, -0.2340,
           0.1657],
         [ 0.3100,  0.2329,  0.0300, -0.2360, -0.0362, -0.2601

In [15]:
seq_model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=80, bias=True)
  (3): Tanh()
  (4): Linear(in_features=80, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=1, bias=True)
)>

In [16]:
seq_model

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=80, bias=True)
  (3): Tanh()
  (4): Linear(in_features=80, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=1, bias=True)
)

In [17]:
[param.shape for param in seq_model.parameters()]

[torch.Size([8, 5]),
 torch.Size([8]),
 torch.Size([80, 8]),
 torch.Size([80]),
 torch.Size([20, 80]),
 torch.Size([20]),
 torch.Size([1, 20]),
 torch.Size([1])]